# 데이콘 매장 별 매출 예측 경진대회

https://www.dacon.io/competitions/official/140472/overview/

이 경진대회에서는 매장 별 (store_id) 결제 내역이 주어지고, 각 매장의 3달 매출('amount')의 합을 예측하는 경진대회입니다.

해당 대회에서는 예측을 위해 단순이동평균 (SMA)과 지수이동평균 (EWMA)가 사용되었습니다.

In [93]:
from google.colab import drive
drive.mount('/content/drive', force_remount = False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [94]:
%cd /content/drive/My Drive/Colab Notebooks/Dacon Competition

/content/drive/My Drive/Colab Notebooks/Dacon Competition


In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [183]:
df = pd.read_csv('funda_train.csv')
df.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6556613 entries, 0 to 6556612
Data columns (total 9 columns):
 #   Column            Dtype  
---  ------            -----  
 0   store_id          int64  
 1   card_id           int64  
 2   card_company      object 
 3   transacted_date   object 
 4   transacted_time   object 
 5   installment_term  int64  
 6   region            object 
 7   type_of_business  object 
 8   amount            float64
dtypes: float64(1), int64(3), object(5)
memory usage: 450.2+ MB


# EDA

In [97]:
# 중복된 결제내역이 있어 제거
df = df.drop_duplicates()

In [ ]:
df.isnull().sum()

store_id                  0
card_id                   0
card_company              0
transacted_date           0
transacted_time           0
installment_term          0
region              2040293
type_of_business    3947728
amount                    0
dtype: int64

In [184]:
print(f'Store Unique:', df['store_id'].nunique())
print(f'Card Unique:', df['card_id'].nunique())
print(f'Card Company Unique:', df['card_company'].nunique())
print('Date min & max:', df['transacted_date'].min(), df['transacted_date'].max())
print('Installment Unique', df['installment_term'].nunique())

Store Unique: 1967
Card Unique: 3950001
Card Company Unique: 8
Date min & max: 2016-06-01 2019-02-28
Installment Unique 34


위에서 결제내역은 2016년 6월부터 2019년 2월까지 있음을 확인할 수 있습니다.

In [98]:
df['full_date'] = pd.to_datetime(df['transacted_date'] + ' ' + df['transacted_time'])
df = df.drop(['region', 'type_of_business'], axis = 1)

In [99]:
df['year'] = df['full_date'].dt.year
df['month'] = df['full_date'].dt.month

# 시계열 데이터 형식으로 전처리

In [100]:
resampled = df.groupby(['store_id', 'year', 'month'])['amount'].sum().reset_index()
resampled[['store_id', 'year', 'month']] = resampled[['store_id', 'year', 'month']].astype(str)
resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60232 entries, 0 to 60231
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   store_id  60232 non-null  object 
 1   year      60232 non-null  object 
 2   month     60232 non-null  object 
 3   amount    60232 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.8+ MB


In [101]:
resampled['month'] = resampled['month'].map(lambda x: '0' + x if len(x) == 1 else x)
resampled['date'] = pd.to_datetime(resampled['year'] + resampled['month'], format = '%Y%m')

매장별로 결제 내역이 없는 날짜가 있어, 1달 기준으로 수합 후 월별 매출을 예측합니다

수합하기 위해 기존 날짜의 일자를 모두 해당 월 마지막 날로 변경해줍니다

In [102]:
def last_day_of_month(date):
  from calendar import monthrange
  return date.replace(day = monthrange(date.year, date.month)[1])

In [103]:
resampled['lastday'] = resampled['date'].map(last_day_of_month)

In [ ]:
resampled.head()

,store_id,year,month,amount,date,lastday
0,0,2016,06,7.470000e+05,2016-06-01,2016-06-30
1,0,2016,07,1.005000e+06,2016-07-01,2016-07-31
2,0,2016,08,8.697143e+05,2016-08-01,2016-08-31
3,0,2016,09,8.978571e+05,2016-09-01,2016-09-30
4,0,2016,10,8.354286e+05,2016-10-01,2016-10-31


In [104]:
resampled = resampled.drop(['year', 'month', 'date'], axis = 1)
resampled = resampled[['store_id', 'lastday', 'amount']]
resampled.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60232 entries, 0 to 60231
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   store_id  60232 non-null  object        
 1   lastday   60232 non-null  datetime64[ns]
 2   amount    60232 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.4+ MB


In [185]:
resampled.head()

,store_id,lastday,amount
0,0,2016-06-30,7.470000e+05
1,0,2016-07-31,1.005000e+06
2,0,2016-08-31,8.697143e+05
3,0,2016-09-30,8.978571e+05
4,0,2016-10-31,8.354286e+05


In [ ]:
resampled.groupby('store_id')['lastday'].count().sort_values()

store_id
795      4
632     12
2119    12
808     13
1041    14
        ..
1911    33
1140    33
1141    33
1940    33
0       33
Name: lastday, Length: 1967, dtype: int64

In [186]:
resampled[resampled['store_id'] == '795']

,store_id,lastday,amount
23189,795,2017-03-31,0.000000
23190,795,2017-04-30,471498.571429
23191,795,2017-05-31,34828.571429
23192,795,2019-02-28,0.000000


위 795번 매장처럼 취소된 건들에 의해 해당 달의 총 매출 합이 0인 경우가 있어, 총 매출이 0인 일자는 모두 지운 데이터프레임을 만들었습니다

In [63]:
resampled_0remove = resampled[resampled['amount'] != 0]

## 취소건 제거 버전
취소가 되었던 결제내역 (amount가 0보다 낮은)을 제거하고, 누적 매출이 예측에 영향을 줄 수 있을거라 생각해 새로운 데이터프레임을 만들었습니다

In [ ]:
df_nocan = df[df['amount'] > 0]

In [74]:
resampled_nocan = df_nocan.groupby(['store_id', 'year', 'month'])['amount'].sum().reset_index()
resampled_nocan[['store_id', 'year', 'month']] = resampled_nocan[['store_id', 'year', 'month']].astype(str)
resampled_nocan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60228 entries, 0 to 60227
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   store_id  60228 non-null  object 
 1   year      60228 non-null  object 
 2   month     60228 non-null  object 
 3   amount    60228 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.8+ MB


In [75]:
resampled_nocan['month'] = resampled_nocan['month'].map(lambda x: '0' + x if len(x) == 1 else x)
resampled_nocan['date'] = pd.to_datetime(resampled_nocan['year'] + resampled_nocan['month'], format = '%Y%m')

In [76]:
resampled_nocan['lastday'] = resampled_nocan['date'].map(last_day_of_month)

In [77]:
resampled_nocan.head()

,store_id,year,month,amount,date,lastday
0,0,2016,06,7.555714e+05,2016-06-01,2016-06-30
1,0,2016,07,1.005000e+06,2016-07-01,2016-07-31
2,0,2016,08,8.715714e+05,2016-08-01,2016-08-31
3,0,2016,09,8.978571e+05,2016-09-01,2016-09-30
4,0,2016,10,8.374286e+05,2016-10-01,2016-10-31


In [78]:
resampled_nocan = resampled_nocan.drop(['year', 'month', 'date'], axis = 1)
resampled_nocan = resampled_nocan[['store_id', 'lastday', 'amount']]
resampled_nocan.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60228 entries, 0 to 60227
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   store_id  60228 non-null  object        
 1   lastday   60228 non-null  datetime64[ns]
 2   amount    60228 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.4+ MB


## 카드별 결제 총합이 음수인 카드 제거
한 매장에서 카드의 총 결제내역이 음수인 경우 환불이 여러번 처리 된 것으로 간주하고 제거

In [ ]:
a = df.groupby(['store_id', 'card_id'])['amount'].sum().sort_values().reset_index()
a

,store_id,card_id,amount
0,572,1108115,-3.428571e+05
1,119,289522,-2.857143e+05
2,119,289448,-2.428571e+05
3,1354,2920340,-2.214286e+05
4,119,290752,-2.142857e+05
...,...,...,...
4067927,538,1024659,3.528571e+07
4067928,538,1024762,3.910024e+07
4067929,1408,3044255,4.431771e+07
4067930,538,1024773,4.555684e+07


In [187]:
df[df['card_id'] == 1108115]

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
1593646,572,1108115,a,2018-04-08,01:31,0,부산 연제구,NaN,342857.142857
1593647,572,1108115,a,2018-04-08,01:31,0,부산 연제구,NaN,-342857.142857
1593648,572,1108115,a,2018-04-08,01:31,0,부산 연제구,NaN,342857.142857
1593652,572,1108115,a,2018-04-09,20:37,0,부산 연제구,NaN,-342857.142857
1593786,572,1108115,a,2018-05-29,00:17,0,부산 연제구,NaN,240000.000000
1593788,572,1108115,a,2018-05-29,20:48,0,부산 연제구,NaN,-240000.000000


In [ ]:
neg_card = a[a['amount'] < 0]['card_id'].values.tolist()

In [ ]:
df['card_id'] = df['card_id'].map(lambda x: 'remove' if x in neg_card else x)
df_noneg = df[df['card_id'] != 'remove']

In [ ]:
df_noneg['card_id'].value_counts()

3820295    2441
1877306     645
2838881     563
3820294     512
2440369     429
           ... 
2904284       1
2904283       1
2904282       1
2904280       1
2297101       1
Name: card_id, Length: 3949840, dtype: int64

In [127]:
resampled_noneg = df_noneg.groupby(['store_id', 'year', 'month'])['amount'].sum().reset_index()
resampled_noneg[['store_id', 'year', 'month']] = resampled_noneg[['store_id', 'year', 'month']].astype(str)
resampled_noneg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60232 entries, 0 to 60231
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   store_id  60232 non-null  object 
 1   year      60232 non-null  object 
 2   month     60232 non-null  object 
 3   amount    60232 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.8+ MB


In [128]:
resampled_noneg['month'] = resampled_noneg['month'].map(lambda x: '0' + x if len(x) == 1 else x)
resampled_noneg['date'] = pd.to_datetime(resampled_noneg['year'] + resampled_noneg['month'], format = '%Y%m')

In [129]:
resampled_noneg['lastday'] = resampled_noneg['date'].map(last_day_of_month)

In [130]:
resampled_noneg.head()

,store_id,year,month,amount,date,lastday
0,0,2016,06,7.470000e+05,2016-06-01,2016-06-30
1,0,2016,07,1.005000e+06,2016-07-01,2016-07-31
2,0,2016,08,8.697143e+05,2016-08-01,2016-08-31
3,0,2016,09,8.978571e+05,2016-09-01,2016-09-30
4,0,2016,10,8.354286e+05,2016-10-01,2016-10-31


In [131]:
resampled_noneg = resampled_noneg.drop(['year', 'month', 'date'], axis = 1)
resampled_noneg = resampled_noneg[['store_id', 'lastday', 'amount']]
resampled_noneg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60232 entries, 0 to 60231
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   store_id  60232 non-null  object        
 1   lastday   60232 non-null  datetime64[ns]
 2   amount    60232 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 1.4+ MB


# Modeling

## 단순이동평균 (Simple Rolling Average)

In [ ]:
resampled.head()

,store_id,lastday,amount
0,0,2016-06-30,7.470000e+05
1,0,2016-07-31,1.005000e+06
2,0,2016-08-31,8.697143e+05
3,0,2016-09-30,8.978571e+05
4,0,2016-10-31,8.354286e+05


In [ ]:
np.append(np.array(resampled[resampled['store_id'] == str(0)]['amount'].values), 0)

array([ 747000.        , 1005000.        ,  869714.28571429,
        897857.14285714,  835428.57142857,  697000.        ,
        761857.14285714,  585642.85714286,  794000.        ,
        720257.14285714,  685285.71428571,  744428.57142857,
        680000.        ,  728285.71428571,  749000.        ,
        840857.14285714,  600571.42857143,  630857.14285714,
        812714.28571429,  643142.85714286,  685285.71428571,
        848428.57142857,  636142.85714286,  686428.57142857,
        707285.71428571,  758714.28571429,  679857.14285714,
        651857.14285714,  739000.        ,  676000.        ,
        874571.42857143,  682857.14285714,  515285.71428571,
             0.        ])

In [188]:
# 한개의 매장에 대한 예측된 3달의 총 매출 합 반환
def rolling_average(win, store_id, df):
  values = np.array(df[df['store_id'] == str(store_id)]['amount'].values)
  for i in range(3):
    new_value = np.mean(values[-win:])
    values = np.append(values, new_value)
  
  result = np.sum(values[-3:])
  return result

In [80]:
rolling_average(6, 0, resampled)

2075355.15873016

In [81]:
# 위 함수를 모든 매장에 대해 적용
def whole_rolling_avg(win_num, df):
  sum = pd.DataFrame(columns = ['store_id', 'sum'])
  for i, store in enumerate(df['store_id'].unique()):
    sum.loc[i] = [store, rolling_average(win_num, str(store), df)]
  sum['store_id'] = sum['store_id'].astype(int)
  return sum

In [17]:
# 단순이동평균의 주기를 설정 후 예측값 데이터프레임을 딕셔너리에 보관
rolling_results = {}
for i in range(2, 13):
  rolling_results[f'sum{i}'] = whole_rolling_avg(i)

In [18]:
rolling_results['sum6']

,store_id,sum
0,0,2.075355e+06
1,1,2.847999e+05
2,2,1.165203e+06
3,4,2.639217e+06
4,5,9.580532e+05
...,...,...
1962,2132,1.954554e+06
1963,2133,5.359897e+05
1964,2134,3.264884e+05
1965,2135,1.648610e+06


# 지수이동평균 (Exponentially Weighted Moving Average)

In [ ]:
resampled[resampled['store_id'] == str(0)].head(5)

,store_id,lastday,amount
0,0,2016-06-30,7.470000e+05
1,0,2016-07-31,1.005000e+06
2,0,2016-08-31,8.697143e+05
3,0,2016-09-30,8.978571e+05
4,0,2016-10-31,8.354286e+05


In [82]:
# 한 매장의 시계열 데이터에 3달 일자를 추가한 후 지수이동평균 계산, 예측값들의 합 반환
def exp_wma(win, store_id, df):
  a = df[df['store_id'] == str(store_id)]
  b = pd.DataFrame({'lastday': ['2019-03-31', '2019-04-30', '2019-05-31']})
  b['lastday'] = pd.to_datetime(b['lastday'])
  c = pd.concat([a, b], axis = 0, join = 'outer', ignore_index = True)
  result = c['amount'].ewm(win, a).mean()[-3:].sum()
  return result

In [86]:
exp_wma(2, 0, resampled_nocan)

1978780.271072408

In [132]:
exp_wma(2, 0, resampled_noneg)

1977775.6572209746

In [83]:
# 위 함수를 모든 매장에 적용
def whole_exp_avg(win_num, df):
  sum = pd.DataFrame(columns = ['store_id', 'sum'])
  for i, store in enumerate(df['store_id'].unique()):
    sum.loc[i] = [store, exp_wma(win_num, str(store), df)]
  sum['store_id'] = sum['store_id'].astype(int)
  return sum

In [87]:
whole_exp_avg(2, resampled_nocan)

,store_id,sum
0,0,1.978780e+06
1,1,2.947969e+05
2,2,1.260239e+06
3,4,2.688657e+06
4,5,9.438694e+05
...,...,...
1962,2132,1.892227e+06
1963,2133,5.237014e+05
1964,2134,4.182125e+05
1965,2135,1.512474e+06


In [21]:
# 매번 비중을 다르게 지정해 값을 계산
exp_wma_result = {}
for i in range(2, 13):
  exp_wma_result[f'exp{i}'] = whole_exp_avg(i, resampled)

In [22]:
exp_wma_result['exp2']

,store_id,sum
0,0,1.977776e+06
1,1,2.899286e+05
2,2,1.259295e+06
3,4,2.667975e+06
4,5,9.412040e+05
...,...,...
1962,2132,1.889129e+06
1963,2133,5.175271e+05
1964,2134,4.163532e+05
1965,2135,1.506335e+06


## 지수이동평균 여러 데이터프레임에 적용
만들어 두었던 기본 시계열 데이터 외 다른 데이터프레임에도 지수이동평균을 적용


In [64]:
exp_wma_result_0remove = {}
for i in range(2, 13):
  exp_wma_result_0remove[f'exp{i}'] = whole_exp_avg(i, resampled_0remove)

In [88]:
exp_wma_result_nocan = {}
for i in range(2, 13):
  exp_wma_result_nocan[f'exp{i}'] = whole_exp_avg(i, resampled_nocan)

In [133]:
exp_wma_result_noneg = {}
for i in range(2, 13):
  exp_wma_result_noneg[f'exp{i}'] = whole_exp_avg(i, resampled_noneg)

In [146]:
def exp_wma_alpha(store_id, df, alpha = 0.33):
  a = df[df['store_id'] == str(store_id)]
  b = pd.DataFrame({'lastday': ['2019-03-31', '2019-04-30', '2019-05-31']})
  b['lastday'] = pd.to_datetime(b['lastday'])
  c = pd.concat([a, b], axis = 0, join = 'outer', ignore_index = True)
  result = c['amount'].ewm(alpha).mean()[-3:].sum()
  return result

## 지수이동평균 가중치 조정
기본 데이터프레임에 지수이동평균을 여러 가중지를 사용해 예측값 계산

In [147]:
def whole_exp_avg_alpha(df, alpha):
  sum = pd.DataFrame(columns = ['store_id', 'sum'])
  for i, store in enumerate(df['store_id'].unique()):
    sum.loc[i] = [store, exp_wma_alpha(str(store), df, alpha = alpha)]
  sum['store_id'] = sum['store_id'].astype(int)
  return sum

In [161]:
exp_wma_result_alpha = {}
for i in np.arange(0.1, 1.1, 0.1):
  exp_wma_result_alpha[f'exp{i}'] = whole_exp_avg_alpha(resampled, alpha = i)

# Submission

In [134]:
submission = pd.read_csv('funda_submission.csv')

In [ ]:
submission.head()

,store_id,amount
0,0,0
1,1,0
2,2,0
3,4,0
4,5,0


In [25]:
rolling_results['sum6']['sum'] * 0.7 + exp_wma_result['exp2']['sum'] * 0.3

0       2.046081e+06
1       2.863385e+05
2       1.193431e+06
3       2.647844e+06
4       9.529984e+05
            ...     
1962    1.934927e+06
1963    5.304510e+05
1964    3.534479e+05
1965    1.605928e+06
1966    6.518564e+06
Name: sum, Length: 1967, dtype: float64

In [58]:
submission['amount'] = exp_wma_result['exp2']['sum']
# submission.loc[submission['store_id'] == 795, 'amount'] = exp_wma_result['exp2'][exp_wma_result['exp2']['store_id'] == 795]['sum'] * 0.2 + rolling_results['sum6'][rolling_results['sum6']['store_id'] == 795]['sum'] * 0.8
submission[submission['store_id'] == 795]
# submission

,store_id,amount
742,795,361810.720879


In [89]:
exp_wma_result_nocan['exp2']['sum']

0       1.978780e+06
1       2.947969e+05
2       1.260239e+06
3       2.688657e+06
4       9.438694e+05
            ...     
1962    1.892227e+06
1963    5.237014e+05
1964    4.182125e+05
1965    1.512474e+06
1966    6.485631e+06
Name: sum, Length: 1967, dtype: float64

In [135]:
submission['amount'] = exp_wma_result_noneg['exp2']['sum']

In [162]:
exp_wma_result_alpha.keys()

dict_keys(['exp0.1', 'exp0.2', 'exp0.30000000000000004', 'exp0.4', 'exp0.5', 'exp0.6', 'exp0.7000000000000001', 'exp0.8', 'exp0.9', 'exp1.0'])

In [181]:
submission['amount'] = whole_exp_avg_alpha(resampled, alpha = 1.95)['sum']
submission

,store_id,amount
0,0,1.971780e+06
1,1,2.901819e+05
2,2,1.261693e+06
3,4,2.669679e+06
4,5,9.348662e+05
...,...,...
1962,2132,1.884030e+06
1963,2133,5.052345e+05
1964,2134,4.165654e+05
1965,2135,1.499356e+06


In [136]:
submission[submission['store_id'] == 795]

,store_id,amount
742,795,290072.175824


In [182]:
submission.to_csv('funda_result.csv', index = False)

# 결과
최종적으로 MAE가 제일 낮았던 예측값은 기본 시계열 데이터와 지수이동평균에 1.95의 가중지를 지정한 모델이었습니다.